# Nigep example using MNIST dataset

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/nigep/*'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.nigep import Nigep
import keras.layers as layers
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers.legacy import RMSprop
from keras.datasets import mnist
import numpy as np

### Load and preprocess MNIST dataset from Keras package

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)  

INPUT_DIMS = x_train.shape[1]
OUTPUT_DIMS = 10

### Build and compile the MLP model

In [3]:
NUM_EPOCHS = 20

model = Sequential()

model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 693962 (2.65 MB)
Trainable params: 693962 (2.65 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

### Execute Nigep to obtain the Generalization Profile results

In [4]:
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_delta=0.001, patience=5, verbose=1)
erl_stopping = EarlyStopping(patience=4, monitor='val_loss', verbose=1)
callbacks = [erl_stopping, lr_reduce]

nigep = Nigep(
    execution_name='mnist',
    x_data=x_train,
    y_data=y_train,
    model=model,
    batch_size=16,
    input_shape=(28,28,1),
    class_mode='categorical',
    k_fold_n=5,
    noise_levels=[0, .2],
    target_names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
    epochs=1,
)

nigep.execute()

Fold: 0 - Training with Noise: 0
3000/3000 [==============================] - 9s 3ms/step - loss: 0.1484 - accuracy: 0.9556
Fold: 0 - Train Noise: 0 - Test Noise: 0


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
nigep.plot_and_save_generalization_profile()